In [17]:
using Plots
using Interact

# Numerical Experiment
## A Comparison of  Lagrange and Barycentric formulas with the Algorithm of Aitken-Neville


In this experiment we extrapolate values $(f(\nu_n^{-2}))_{n=0:N}$ of some test functions for different subdividing sequences $(\nu_{n})$. We compare the convergence behaviour of four algorithms (Lagrange interpolation, barycentric interpolation 1 and 2 and the algorithm of Aitken-Neville). 

In [18]:
# functions we will need
# 1. a function to precompute all necessary weights  
function precomputeValues(N::Integer, sequence::String)
    # initialize subdividing sequence for barycentric weights
    if sequence == "Harmonic"
        subdividingSequence = [BigInt(n+1) for n = 0:N]
    elseif sequence == "Romberg"
        subdividingSequence = [BigInt(2)^n for n = 0:N]
    elseif sequence == "Bulirsch"
        subdividingSequence = [n==0 ? BigInt(1) : (isodd(n) ? BigInt(2)^BigInt(n/2+0.5) : 3*BigInt(2^BigInt(n/2-1))) for n = 0:N]
    end

    # compute nodes corresponding to the subdividing sequence 
    nodes = BigInt(1).// subdividingSequence.^2

    # compute barycentric weights,
    # cf. Berrut and Trefethen. "Barycentric Lagrange Interpolation". DOI: 10.1137/S0036144502417715
    ω = zeros(Rational{BigInt},N+1,N+1)
    ω[1,:] = ones(Rational{BigInt},N+1)
    #for n = 1:N+1
    for n = 2:N+1
        distance = nodes[1:n-1] .- nodes[n]
        ω[1:n-1,n] = ω[1:n-1,n-1] .// distance
        ω[n,n] = one(BigInt) // prod(-distance)
    end

    #rescale barycentric weights
    broadcast!(//, ω, ω, -nodes)
    
    # compute scaling factor for extrapolation operator
    ρ = [prod(-nodes[1:n+1]) for n = 0:N]

    # rescale weights and nodal polynomial
    # again according to Berrut and Trefethen
    c = ( (1 .- nodes).//4 ).^(0:N)
    ρRescaled = ρ .// c
    ωRescaled = broadcast(*,ω,c')
    
    return (nodes,ω,ρ,ωRescaled,ρRescaled)
end

# 2. The algorithms we want to test
# extrapolation with  1. barycentric formula
function B1(data, nodes, ρ, ω, T)
    return T.(ρ) .* vec( T.(data)'*T.(ω) )
end

# extrapolation with  2. barycentric formula
function B2(data, nodes, ρ, ω, T)
    return  vec( T.(data)'*T.(ω) ) ./ T.( vec(sum(ω;dims=1)) )
end

# extrapolation with Lagrange formula
function L(data, nodes, ρ, ω, T)
    w = broadcast(*,ω,ρ')
    return vec( T.(data)'*T.(w) )
end

# extrapolation with algorithm of Aitken Neville
function AN(data, nodes, ρ, ω, T)
    N = length(nodes)-1
    P = zeros(T,N+1) # storage for the tableau
    d = zeros(T,N+1) 
    nodes = T.(nodes)
    P[:] = T.(data[:])
    for m = 1:N
        range = (m+1):(N+1)
        d[range] = -nodes[range] ./( nodes[range] .- nodes[1:N-m+1])
        P[range] = P[range] .+ d[range] .* (P[range] - P[range .- 1])
    end
    return  P
end;

In [19]:
# the options and parameters (including information for the plots)
N = 100 # maximal order of extrapolation

sequenceType = [ "Harmonic", "Romberg","Bulirsch"]
types = [ Float32, Float64, BigFloat]

algorithms = [AN, L, B1, B2]
algorithmsStr = ["Aitken Neville", "Lagrange formula", "Barycentric formula (1)", "Barycentric formula (2)"]
algorithmsColor = [:red, :green, :blue, :yellow ]
algorithmsLinewidth = [1, 1, 3, 1]
scalingOptions = [true, false]
scalingOptionsStr = ["Scaling", "No Scaling"]

testFunctions=[x->cos(40*acos(2*x-1)), x-> 1/(1 +(2*x-1)^2), x->sin(x)] # partially fitted to the interval [0,1]
testFunctionsStr = ["Chebyshev (40)","Runge's function","sine"];

In [20]:
# computations
# 1. extrapolation
results = Dict{DataType,Array}() # storage for the results
for T in types
    results[T] = zeros(T,
        length(testFunctions),
        length(sequenceType),
        length(algorithms),
        length(scalingOptions),
        N+1)
end
preCompValues = Dict( [ (sequence, precomputeValues(N, sequence)) for sequence in sequenceType] ) # precompute weights etc.

for T in types,
    (n1, f) in enumerate(testFunctions),
    (n2, sequence) in enumerate(sequenceType),
    (n3, alg) in enumerate(algorithms),
    (n4, scalingMode) in enumerate(scalingOptions)
    
    # setup
    (nodes,ω,ρ,ωRescaled,ρRescaled) = preCompValues[sequence]
    if scalingMode
        ω, ρ = ωRescaled, ρRescaled
    end
    data = f.(nodes)
    solution = T(f(0))
    
    #computation
    results[T][n1, n2, n3, n4, :] = abs.( alg(data, nodes, ρ, ω,  T) .- solution )
end

# 2. condition numbers
conditionNumber = zeros(Rational{BigInt}, length(sequenceType), N+1) #storage for condition numbers
for (n2, sequence) in enumerate(sequenceType)
    (_, ω, ρ, _, _) = preCompValues[sequence]
    conditionNumber[n2,:] = abs.(ρ) .* vec( sum(abs.(ω); dims=1) )
end

In [21]:
# plot the results
m = @manipulate for T = types,f =testFunctionsStr,s = sequenceType, Rescale = false, ConditionNumber = false
        n1 = findfirst(x->x==f,testFunctionsStr)
        n2 = findfirst(x->x==s,sequenceType)
        n4 = findfirst(x->x==Rescale, scalingOptions)
        # makeup
        plt = plot(xlabel = "#nodes", ylabel = "abs. error", size = (900,300), legend = :top)
        if ConditionNumber
             plot!(plt,1:N+1,eps(T)*conditionNumber[n2,:], linewidth = 2, linestyle = :dot, label = "Condition Number")
        end
        for (n3,name) in enumerate(algorithmsStr)
            err = max.(results[T][n1,n2,n3,n4,:],eps(T))
            plot!(plt,1:N+1,err ,
                    yscale = :log10, label = n3>2 ? name : "",
                    linewidth = algorithmsLinewidth[n3],
                    color = algorithmsColor[n3])
            if n3 == 1
                #special markers for Aitken-Neville
                tAN = 1:5:N+1
                plot!(plt,tAN,err[tAN], label = name,
                    linealpha = 0, markershape = :circle,markersize =4, color = algorithmsColor[n3])
            end
            if n3 ==2
                #special marker for Lagrange
                tL = 1:8:N+1
                 plot!(plt,tL,err[tL], label = name,
                    linealpha = 0, markershape = :star5 ,markersize =5, color = algorithmsColor[n3])
            end
        end
    plt
end

Widget{:manipulate,Any}(OrderedDict{Symbol,Any}(:T=>Widget{:togglebuttons,Any}(OrderedDict{Symbol,Any}(:options=>Observable{Array{DataType,1}} with 2 listeners. Value:
DataType[Float32, Float64, BigFloat],:index=>Observable{Any} with 2 listeners. Value:
1,:vals2idxs=>Observable{Any} with 1 listeners. Value:
DataType[Float32, Float64, BigFloat]), Observable{Any} with 2 listeners. Value:
Float32, Scope("knockout-component-aff5f33e-3c89-4139-a8e7-ccee674a9445", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["T"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"text-align:right;width:18%")), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :button), Any[Node{DOM}(DOM(:html, :label), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text : key")), 0)], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"click: function () {\$root.index(val)}, css: {'is-primary is-selected' : \$root.index() == val, 'button is-fullwidth' : true}")), 1)], Dict{Symbol,Any}(:attributes=>("data-bind"=>"foreach : options_js"),:className=>"field is-grouped has-addons is-oneline is-centered"), 2)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"flex-grow:1; margin: 0 2%")), 3), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[], Dict{Symbol,Any}(), 0)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"width:18%")), 1)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"display:flex; justify-content:center; align-items:center;")), 9), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("options_js"=>(Observable{Any} with 1 listeners. Value:
OrderedDict[OrderedDict{String,Any}("key"=>"Float32","val"=>1,"id"=>"id94ZmRbn0"), OrderedDict{String,Any}("key"=>"Float64","val"=>2,"id"=>"idwN5q6GML"), OrderedDict{String,Any}("key"=>"BigFloat","val"=>3,"id"=>"idMkv3N2TB")], nothing),"index"=>(Observable{Any} with 2 listeners. Value:
1, nothing)), Set(String[]), nothing, Any["knockout"=>"/Users/konstantinalthaus/.julia/packages/Knockout/JIqpG/src/../assets/knockout.js", "knockout_punches"=>"/Users/konstantinalthaus/.julia/packages/Knockout/JIqpG/src/../assets/knockout_punches.js", "/Users/konstantinalthaus/.julia/packages/InteractBase/Q4IkI/src/../assets/all.js", "/Users/konstantinalthaus/.julia/packages/InteractBase/Q4IkI/src/../assets/style.css", "/Users/konstantinalthaus/.julia/packages/InteractBulma/Ohu5Y/src/../assets/main.css"], Dict{Any,Any}("_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = JSON.parse(\"{\\\"options_js\\\":[{\\\"key\\\":\\\"Float32\\\",\\\"val\\\":1,\\\"id\\\":\\\"id94ZmRbn0\\\"},{\\\"key\\\":\\\"Float64\\\",\\\"val\\\":2,\\\"id\\\":\\\"idwN5q6GML\\\"},{\\\"key\\\":\\\"BigFloat\\\",\\\"val\\\":3,\\\"id\\\":\\\"idMkv3N2TB\\\"}],\\\"index\\\":1}\");\n    var self = this;\n    function AppViewModel() {\n        for (var key in json_data) {\n            var el

This plot shows the absolute error $|f(0)-\pi_N(0)|$ in dependence of the number of nodes $N$ used for intetpolation. $\pi_N$ is the interpolant of the values $(f(\nu_n^{-2}))_{n=0:N}$ in the nodes $(\nu_n^{-2})_{n=0:N}$.
The line labeled "condition number" in the plot is the graph of  $N \mapsto \|\mathcal{E}_N \|_\infty \text{eps}$. Here $\text{eps}$ is the machine precission of the selected `Float` representation.

## Interpretation of the results

The convergence behaviour of all algorithms is satisfactory except if the harmonic sequence is used. In that case the absolute error at first decreases as $N$ grows but after some time increases dramatically. In particular all algorithms save the one due to Aitken-Neville produce an error that is oscillating between several orders of magnitude, i.e. only noise is produced.

This behaviour is not due to numerical instabilities of the specific implementations used here. The reason is that the problem itself is ill conditioned.

The erratic behaviour in the plot begins shortly after the convergence plot intersects the graph of $N \mapsto \|\mathcal{E}_N \|_\infty \text{eps}$.
The latter function is - due to the linearity of interpolation - an upper bound of the error we would expect if we computed $\pi_N(0)$ in exact arithmetic on basis of disturbed values $(f(\nu_n^{-2})+\epsilon_n)_{n=0:N}$ with a disurbance that is smaller than the machine precision $\text{eps}$ ($|\epsilon_n| \leq \text{eps}$).

Thus if $N$ is large enough, the unavoidable and problem inherent amplification of the rounding errors "takes over" and we observe divergence